In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import tools, batch

In [3]:
import ipygee as ui

In [4]:
import datetime
import math

In [5]:
Map = ui.Map()
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href…

In [6]:
provincias = ee.FeatureCollection('users/rprincipe/norte_argentina')

In [7]:
def convert_provincias(feat):    
    n = ee.String(feat.get('NAM'))

    n = ee.String(ee.Algorithms.If(n.compareTo('Córdoba'), n, 'Cordoba'))
    n = ee.String(ee.Algorithms.If(n.compareTo('San Juan'), n, 'San_Juan'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Santa Fe'), n, 'Santa_Fe'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Santiago del Estero'), n, 'Santiago_del_Estero'))
    n = ee.String(ee.Algorithms.If(n.compareTo('Tucumán'), n, 'Tucuman'))

    return feat.set('NAME', ee.String(n))

In [8]:
provincias = provincias.map(convert_provincias)

In [9]:
def get_province(fc, field, name):
    return ee.Feature(fc.filterMetadata(field, 'equals', name).first())

In [10]:
Map.addLayer(provincias, name='Provincias')

In [11]:
maxFeatures = 3 # x 100

In [12]:
# alerts
alerts = ee.ImageCollection('projects/glad/alert/UpdResult')

In [13]:
# get rid of projects/glad/alert/UpdResult/01_01_SBRA
def rid_SBRA(img, l):
    l = ee.List(l)
    theid = img.id()
    return ee.Algorithms.If(theid.compareTo('01_01_SBRA'), l.add(img), l)

alerts = ee.List(alerts.iterate(rid_SBRA, ee.List([])))
alerts = ee.ImageCollection.fromImages(alerts)

In [14]:
# change field
change_fld = 'conf19'

country_alerts = alerts.filterBounds(provincias.geometry())

today = datetime.date.today()

scale = 30

In [48]:
def get_days(collection, month, year):
    def wrap(img):
        d = img.date()
        m = d.get('month')
        y = d.get('year')
        return img.set('MONTH', m, 'YEAR', y)
    
    col_date = collection.map(wrap)

    filtered = col_date.filterMetadata('MONTH', 'equals', month).filterMetadata('YEAR', 'equals', year)
    
    def iteration(img, l):
        l = ee.List(l)
        img = ee.Image(img)
        d = img.date().get('day')
        return l.add(d)
    
    days = ee.List(filtered.iterate(iteration, ee.List([]))).sort()
    days_str = days.map(lambda d: ee.Number(d))

    return days_str.distinct()

In [49]:
ui.eprint(get_days(alerts, 6, 2019))

In [16]:
def get_pixel_limit(area, scale=30):
    return math.floor((area*10000)/(scale*scale))

In [17]:
def confirmed_mask(collection):
    last2 = collection.limit(2, 'system:time_start', False)
    last2list = last2.toList(2)
    
    last_img = ee.Image(last2list.get(0))
    last_conf = last_img.select('conf19')
    
    before_img = ee.Image(last2list.get(1))
    before_conf = before_img.select('conf19')

    confirmed = last_conf.eq(3).And(before_conf.eq(2)).rename('confirmed')

    doy = last_img.select('alertDate19').rename('alert_doy')

    return last_img.addBands([confirmed, doy]).updateMask(confirmed).unmask()

In [18]:
def probable_mask(collection):
    last2 = collection.limit(2, 'system:time_start', False)
    last2list = last2.toList(2)
    
    last_img = ee.Image(last2list.get(0))
    last_mask = last_img.select('conf19')
    
    before_img = ee.Image(last2list.get(1))
    before_mask = before_img.select('conf19')
    
    new_probable = last_mask.eq(2).And(before_mask.eq(0)).rename('probable')
    
    return last_img.addBands(new_probable).updateMask(new_probable).unmask()

In [19]:
def get_rid_min_area(bool_image, limit):
    # Returns a mask

    # get connected pixels
    conn = bool_image.connectedPixelCount(1000).rename('connected')

    # add mask and connected
    lossconn = bool_image.addBands(conn)

    # get holes and islands
    island = bool_image.eq(1).And(conn.lte(limit))
    holes = bool_image.eq(0).And(conn.lte(limit))

    # fill holes
    filled = bool_image.where(holes, 1)
    # get rid island
    no_island = filled.where(island, 0)

    return no_island

In [20]:
def make_smooth(image, alg):
    if (alg == 'no smooth'):
        result = image
    elif (alg == 'max'):
        result = image.focal_max()
    elif (alg == 'mode'):
        result = image.focal_mode()

    return result.set('system:time_start', image.date().millis())

In [63]:
def histogram(alert, clas, region=None):
    
    if not region:
        region = alert.geometry()
    
    result = alert.select(clas).reduceRegion(**{
        'reducer': ee.Reducer.fixedHistogram(0, 2, 2),
        'geometry': region,
        'scale': alert.projection().nominalScale(),
        'maxPixels': 1e13
    })
    
    result = result.get(clas)

    count = ee.Number(ee.Algorithms.If(result, ee.Array(result).get([1, 1]),0))

    return count

In [22]:
def make_vector(image, region):
    reducer = ee.Reducer.max()
    vector = image.reduceToVectors(**{
        'geometry': region,
        'reducer': reducer,
        'scale': image.select([0]).projection().nominalScale(),
        'maxPixels': 1e13
    })
    return vector

In [23]:
def get_probable(collection, site, date, limit=1, smooth='max'):
    date = ee.Date(date)
    
    # filter collection up to selected date
    start = ee.Date.fromYMD(d.get('year'), 1, 1)
    col = collection.filterDate(ee.Date(start), date.advance(1, 'day'))
    
    col = col.filterBounds(site)

    # get loss
    loss = probable_mask(col)

    # loss mask
    loss_mask = loss.select('probable')

    # observation date
    alertDate = loss.select('alertDate19')

    # get rid of min area
    final_mask = get_rid_min_area(loss_mask, limit)

    # smooth
    final_mask = make_smooth(final_mask, smooth)

    # add doy band
    smooth_alertDate = make_smooth(alertDate, 'max').rename('alert_doy')

    # add date band
    dateBand =  tools.image.doyToDate(smooth_alertDate).rename('alert_date')

    final = final_mask.addBands([smooth_alertDate, dateBand]).updateMask(final_mask)

    return final

In [24]:
def dateFromDatetime(dt):
    return ee.Date(dt.isoformat())

In [25]:
d = dateFromDatetime(today)

In [26]:
def get_confirmed(collection, site, date, limit=1, smooth='max'):
    date = ee.Date(date)
    
    # filter collection up to selected date
    start = ee.Date.fromYMD(date.get('year'), 1, 1)
    col = country_alerts.filterDate(ee.Date(start), date.advance(1, 'day'))
    
    col = col.filterBounds(site)
    
    # get mask
    loss = confirmed_mask(col)

    # get rid of min area
    final_mask = get_rid_min_area(loss.select('confirmed'), limit)

    # smooth
    final_mask = make_smooth(final_mask, smooth)

    # mask doy
    doy = loss.select('alert_doy')
    
    # focal_max doy
    smooth_alertDate = make_smooth(doy, smooth)

    # add date band
    dateBand =  tools.image.doyToDate(smooth_alertDate).rename('alert_date')

    return final_mask.addBands([dateBand, smooth_alertDate]).updateMask(final_mask)

In [27]:
tuc = get_province(provincias, 'NAME', 'Tucuman')

In [28]:
prob = get_probable(alerts, tuc.geometry(), '2019-04-20')

In [29]:
Map.addLayer(prob, name='probable')

In [64]:
ui.eprint(histogram(prob, 'probable', tuc.geometry()))

In [30]:
def download_alert(collection, clas, date, region, min_area=1, smooth='max', max_features=3):
    
    funcs = {'probable': get_probable, 'confirmed': get_confirmed}
        
    limit = get_pixel_limit(min_area)
    alert = funcs[clas](collection, region, date, limit, smooth)

    count = histogram(alert, clas, region)

    def download(i):

        vector = ee.FeatureCollection(ee.Algorithms.If(count,
        make_vector(alert, region), ee.FeatureCollection([])))

        vlist = vector.toList(100, i*100)

        v = ee.FeatureCollection(vlist)

        msg = '{}_alert_for_{}_{}'.format(clas, date, i)
        no_msg = 'No {} alerts for {} {}'.format(clas, date, i)

        name = ee.String(ee.Algorithms.If(v.size(), msg, no_msg))
        
        name_cli = name.getInfo()
        
        print(name_cli)
        if name_cli != no_msg:
            url = vector.getDownloadURL(**{
                'filetype': 'json', 
                'filename': name_cli
            })
            print(url)

    for i in range(max_features+1):
        download(i)

In [32]:
download_alert(alerts, 'probable', '2019-06-11', tuc.geometry())

No probable alerts for 2019-06-11 0
No probable alerts for 2019-06-11 1
No probable alerts for 2019-06-11 2
No probable alerts for 2019-06-11 3


In [34]:
ui.eprint(get_days(alerts, 6, 2019))

In [36]:
datetime.datetime.today().isoformat()

'2019-06-10T08:43:29.627022'

In [52]:
r = ee.Image.random(1)
rm = r.gt(0.8)
r = r.updateMask(rm)

In [55]:
Map.addLayer(r, {'palette':['red']}, name='random red')

In [56]:
test = ee.Image.focal_max(r)

In [57]:
Map.addLayer(test, {'palette':['orange']}, name='test')

In [67]:
year = ee.String('2019')

In [69]:
ui.eprint(year.slice(2))